In [2]:
import pickle

model = pickle.load(open("models/cifar100/vgg/vgg16_cifar100_focuseddropout0.1", "rb"))

In [4]:
model.state_dict()

OrderedDict([('features.0.weight',
              tensor([[[[ 2.8495e-03,  5.5961e-02,  9.6505e-03],
                        [ 3.4472e-02,  9.1884e-03,  1.5905e-02],
                        [-4.6293e-02, -1.0177e-01, -3.5880e-02]],
              
                       [[-3.4865e-02,  2.4220e-02, -1.5032e-02],
                        [-7.2226e-03,  2.1193e-02,  2.1246e-02],
                        [-9.9255e-03, -2.3825e-02,  6.6979e-02]],
              
                       [[ 6.1653e-02,  1.0074e-03, -8.2919e-02],
                        [ 4.3495e-02, -3.4924e-02, -7.4555e-02],
                        [-7.9721e-03, -6.3261e-03, -7.2617e-05]]],
              
              
                      [[[ 5.6970e-02,  5.9784e-02, -1.9295e-01],
                        [ 4.3067e-02,  9.2950e-02, -1.4349e-01],
                        [ 6.5351e-03,  9.0979e-02, -3.2837e-02]],
              
                       [[ 2.8939e-02,  1.1531e-01, -3.3999e-02],
                        [-5.1296e-02,  1

In [6]:
import torch
filename = 'vgg16_cifar100_focuseddropout0.1.pth'
torch.save(model.state_dict(), filename)

In [44]:
import glob
import cv2
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

def get_places365(train = True):
    if train:
        imgs_path = "/datasets-2/places365/places365_standard/train/"
    else:
        imgs_path = "/datasets-2/places365/places365_standard/val/"
    
    file_list = glob.glob(imgs_path + "*")
    lst = []
    class_names = set()
    
    for class_path in file_list:
        class_name = class_path.split("/")[-1]
        class_names.add(class_name)
        for img_path in glob.glob(class_path + "/*.jpg"):
            lst.append([img_path, class_name])
    return lst, sorted(class_names)

def getitem(d, class_map):
    # Get image
    img = cv2.imread(d[0])
    img_tensor = torch.from_numpy(img).float()
    img_tensor = img_tensor.permute(2, 0, 1)
    
    # Get class_id (0-364)
    class_name = d[1]
    class_id = class_map[class_name]
    class_id = torch.tensor([class_id]).item()
    
    return img_tensor, class_id

In [2]:
from utils.vgg16 import VGG
import torch
import torch.nn as nn


device = 'cuda'
model = VGG('VGG16', num_classes = 365)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

train_losses, train_accs, val_losses, val_accs = [], [], [], []

print("Model setup")

Model setup


In [45]:
train_data, class_names = get_places365(train = True)
test_data, _ = get_places365(train = False)
class_names = sorted(class_names)
class_map = {c:i for i, c in zip(range(len(class_names)), class_names)}

In [46]:
class_names

['airfield',
 'airplane_cabin',
 'airport_terminal',
 'alcove',
 'alley',
 'amphitheater',
 'amusement_arcade',
 'amusement_park',
 'apartment_building-outdoor',
 'aquarium',
 'aqueduct',
 'arcade',
 'arch',
 'archaelogical_excavation',
 'archive',
 'arena-hockey',
 'arena-performance',
 'arena-rodeo',
 'army_base',
 'art_gallery',
 'art_school',
 'art_studio',
 'artists_loft',
 'assembly_line',
 'athletic_field-outdoor',
 'atrium-public',
 'attic',
 'auditorium',
 'auto_factory',
 'auto_showroom',
 'badlands',
 'bakery-shop',
 'balcony-exterior',
 'balcony-interior',
 'ball_pit',
 'ballroom',
 'bamboo_forest',
 'bank_vault',
 'banquet_hall',
 'bar',
 'barn',
 'barndoor',
 'baseball_field',
 'basement',
 'basketball_court-indoor',
 'bathroom',
 'bazaar-indoor',
 'bazaar-outdoor',
 'beach',
 'beach_house',
 'beauty_salon',
 'bedchamber',
 'bedroom',
 'beer_garden',
 'beer_hall',
 'berth',
 'biology_laboratory',
 'boardwalk',
 'boat_deck',
 'boathouse',
 'bookstore',
 'booth-indoor',
 'b

In [ ]:
from tqdm import tqdm
import numpy as np

def loader(data, indices, i, batch_size, class_map):
    n = len(data)
    return DataLoader([getitem(data[indices[idx]], class_map) for idx in range(i, min(n, i+batch_size))], batch_size = 128)

def train(model, train_data, optimizer, criterion, device, class_map):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    batch_size = 128
    n = len(train_data)
    indices = np.random.permutation(n)
    
    for i in tqdm(range(0, n, batch_size), total = n//batch_size, position = 0):
        trainloader = loader(train_data, indices, i, batch_size, class_map)
        for data in trainloader:
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            train_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += (preds == labels).sum().item()
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()

    # Loss and accuracy for the complete epoch.
    epoch_loss = train_running_loss / counter
    epoch_acc =  (train_running_correct / n)
    return epoch_loss, epoch_acc

def validate(model, val_data, criterion, device, class_map):
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    batch_size = 128
    n = len(val_data)
    indices = range(n)
    
    with torch.no_grad():
        for i in tqdm(range(0, n, batch_size), total = n//batch_size, position = 0):
            validloader = loader(val_data, indices, i, batch_size, class_map)
            for data in validloader:
                counter += 1
                image, labels = data
                image = image.to(device)
                labels = labels.to(device)
                # Forward pass.
                outputs = model(image)
                # Calculate the loss.
                loss = criterion(outputs, labels)
                valid_running_loss += loss.item()
                # Calculate the accuracy.
                _, preds = torch.max(outputs.data, 1)
                valid_running_correct += (preds == labels).sum().item()
        
    # Loss and accuracy for the complete epoch.
    epoch_loss = valid_running_loss / counter
    epoch_acc = (valid_running_correct / n)
    return epoch_loss, epoch_acc

In [43]:
epochs = 100
for _ in range(epochs):
    print(f"Epoch {_}")
#     train_loss, train_acc = train(model, train_data, optimizer, criterion, device, class_map)
#     train_losses.append(train_loss)
#     train_accs.append(train_acc)
    
    val_loss, val_acc = validate(model, test_data, criterion, device, class_map)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    scheduler.step()
    print(f"Training loss: {train_loss}, Training accuracy: {train_acc}")
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")
    
print("Training Done!")
print("Best training accuracy: ", max(train_accs))
print("Best validation accuracy: ", max(val_accs))
# plot(train_losses, train_accs, val_losses, val_accs, model_name)

Epoch 0


 39%|███▊      | 147/380 [03:39<05:48,  1.49s/it]


KeyboardInterrupt: 

In [ ]:
########################################Train & Validate#####################################################
epochs = 100
train_loader = Places365(batch_size=32)
print("Retrieved Data")

Retrieving Test Data


 44%|████▍     | 15987/36500 [05:53<08:04, 42.36it/s]

In [ ]:
model.to(device)
loss = nn.CrossEntropyLoss()
loss.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

train_losses, train_accs, val_losses, val_accs = [], [], [], []

print("Model setup")

for _ in range(epochs):
    print(f"Epoch {_}")
    train_loss, train_acc = train(model, train_loader, optimizer, loss, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    val_loss, val_acc = validate(model, valid_loader, loss, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    scheduler.step()
    print(f"Training loss: {train_loss}, Training accuracy: {train_acc}")
    print(f"Validation loss: {val_loss}, Validation accuracy: {val_acc}")
print("Training Done!")
print("Best training accuracy: ", max(train_accs))
print("Best validation accuracy: ", max(val_accs))
plot(train_losses, train_accs, val_losses, val_accs, model_name)

# Save Model
pickle.dump(model, open(filename, "wb"))
print(f"Model saved on {filename}")

Model setup
Epoch 0


  0%|          | 0/1141 [00:00<?, ?it/s]

In [1]:
from focuseddropout import FocusedDropout

In [2]:
import torch.nn.functional as F
import torch.nn as nn
import torch
from torch import Tensor
from typing import Type

class BasicBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1, expansion: int = 1, downsample: nn.Module = None, dropout_rate: float = 0.2) -> None:
        super(BasicBlock, self).__init__()
        # Multiplicative factor for the subsequent conv2d layer's output channels.
        # It is 1 for ResNet18 and ResNet34.
        self.expansion = expansion
        self.downsample = downsample
        self.conv1 = nn.Conv2d(
            in_channels, 
            out_channels, 
            kernel_size=3, 
            stride=stride, 
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.drop = FocusedDropout(par_rate = 0.1)
        self.conv2 = nn.Conv2d(
            out_channels, 
            out_channels*self.expansion, 
            kernel_size=3, 
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)
        
    def forward(self, x: Tensor) -> Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.drop(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        out = self.drop(out)
        return  out
    
class ResNet(nn.Module):
    def __init__(self, img_channels: int, num_layers: int, block: Type[BasicBlock], num_classes: int  = 1000, dropout_rate: float = 0.2) -> None:
        super(ResNet, self).__init__()
        if num_layers == 18:
            # The following `layers` list defines the number of `BasicBlock` 
            # to use to build the network and how many basic blocks to stack
            # together.
            layers = [2, 2, 2, 2]
            self.expansion = 1
        
        self.in_channels = 64
        # All ResNets (18 to 152) contain a Conv2d => BN => ReLU for the first
        # three layers. Here, kernel size is 7.
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7, 
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.drop = FocusedDropout(par_rate = 0.1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*self.expansion, num_classes)
        
        
    def _make_layer(self, block: Type[BasicBlock], out_channels: int, blocks: int, stride: int = 1) -> nn.Sequential:
        downsample = None
        if stride != 1:
            """
            This should pass from `layer2` to `layer4` or 
            when building ResNets50 and above. Section 3.3 of the paper
            Deep Residual Learning for Image Recognition
            (https://arxiv.org/pdf/1512.03385v1.pdf).
            """
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels, 
                    out_channels*self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False 
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = []
        layers.append(
            block(
                self.in_channels, out_channels, stride, self.expansion, downsample
            )
        )
        self.in_channels = out_channels * self.expansion
        for i in range(1, blocks):
            layers.append(block(
                self.in_channels,
                out_channels,
                expansion=self.expansion
            ))
        return nn.Sequential(*layers)
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # The spatial dimension of the final layer's feature 
        # map should be (7, 7) for all ResNets.
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [6]:
from getdata import get_data
from train import train, validate
import torch.optim as optim
import torch
from torch.utils.data import DataLoader

In [2]:
train_loader, valid_loader = get_data(batch_size = 1)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
i = 0
lst = []
for image, label in valid_loader:
    if i < 128:
        lst.append((image[0], label.item()))
        i+=1
    else:
        break

In [8]:
torch.save(lst, "test_data.pt")